<a href="https://colab.research.google.com/github/ElederSan/lab-nlp/blob/master/solved_challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

In [3]:
pip install stop-words

# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets.

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.

Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:*

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/archive (2)/training.1600000.processed.noemoticon.csv",
                   on_bad_lines='skip', sep=",", encoding='ISO-8859-1')
data.columns=  ['target','id','date','flag','user','text']
data = data.sample(n=20000, random_state=42).reset_index(drop=True)

In [6]:
data.head()

,target,id,date,flag,user,text
0,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...
1,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...
2,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...
3,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...
4,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick"


In [7]:
data.shape

(20000, 6)

In [8]:
data["target"].unique()

array([0, 4])

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [9]:
# your code here

In [10]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words
import nltk
nltk.download('words')
from nltk.corpus import stopwords
from stop_words import get_stop_words

def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    s = s.lower()
    s = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ', s)
    s= re.sub(r'\d+',' ',s) # Replace one or more digits by  ' '
    s = re.sub(r'\W+',' ',s) # Replace everything non-alpahnumeric by ' '
    s = BeautifulSoup(s, 'lxml').get_text().strip()

    return s


def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    s = word_tokenize(s)

    return s

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    snow_stemmer = SnowballStemmer(language='english')

    stemmed_list = [snow_stemmer.stem(word) if snow_stemmer.stem(word) in words.words() else word for word in l]

    lem = WordNetLemmatizer()

    stem_lemm_list=[lem.lemmatize(word) for word in stemmed_list]

    return stem_lemm_list

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = set(stopwords.words("english"))
    list_wo_stop = [word for word in l if word.lower() not in stop_words]
    return list_wo_stop

    #stop_words = get_stop_words('en')

    # Clean stop words
    #stop_words_clean = [re.sub(r"\s*'\s*\w*", "", word) for word in stop_words]

    #list_wo_stop = [word for word in l if word not in stop_words_clean]

    #return list_wo_stop


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [11]:
data["text"] = data["text"].apply(lambda x: clean_up(x))

In [12]:
data["text_processed"] = data["text"].apply(lambda x: tokenize(x)) # We leave it without running, as it´s countinously breaking

In [13]:
stem_and_lemmatize_vec = np.vectorize(stem_and_lemmatize)

In [14]:
#from tqdm import tqdm

#tqdm.pandas(desc="Processing", leave=False)
#data["text_processed"]=data["text_processed"].progress_apply(stem_and_lemmatize)
#data.head()


In [15]:
# data.to_csv('/content/drive/MyDrive/data_tweet.csv', index=False)


In [16]:
data["text_processed"] = data["text_processed"].apply(lambda x: remove_stopwords(x))

In [17]:
# data = pd.read_csv("/content/drive/MyDrive/data_tweet.csv")

In [18]:
data.head()

,target,id,date,flag,user,text,text_processed
0,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,nkluvr eva my poor little dumpling in holmdel ...,"[nkluvr, eva, poor, little, dumpling, holmdel,..."
1,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,i m off too bed i gotta wake up hella early to...,"[bed, got, ta, wake, hella, early, tomorrow, m..."
2,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,i havent been able to listen to it yet my spea...,"[havent, able, listen, yet, speakers, busted]"
3,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,"[remembers, solving, relatively, big, equation..."
4,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,ate too much feel sick,"[ate, much, feel, sick]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [19]:
corpus = data['text_processed'].apply(lambda x: ' '.join(x))  # Convert the list of words to a space-separated string

In [20]:
corpus

0        nkluvr eva poor little dumpling holmdel vids r...
1             bed got ta wake hella early tomorrow morning
2                   havent able listen yet speakers busted
3        remembers solving relatively big equation two ...
4                                       ate much feel sick
                               ...                        
19995                                    girlwonder thanks
19996    trying study biggest test next week working th...
19997             finished watching song presents boystown
19998                            janfran awww wait get one
19999                                              nothing
Name: text_processed, Length: 20000, dtype: object

In [22]:
wordfreq = {} # A dictionary to hold as keys every single word, and the value is how many times appears the word.
for sentence in corpus:
    words = sentence.split() # List of words in a given review.
    #tokens = nltk.word_tokenize(sentence) # To get the words, it can be also done with sentence.split()
    for word in words:
        if ( word not in wordfreq.keys() ):
            wordfreq[word] = 1 # We initialize the corresponding counter
        else:
            wordfreq[word] += 1 # We increase the corresponding counte

In [23]:
wordfreq = dict(sorted(wordfreq.items(), key=lambda x: x[1], reverse=True))
wordfreq

{'good': 1173,
 'day': 1105,
 'get': 1042,
 'like': 969,
 'quot': 922,
 'go': 915,
 'love': 856,
 'got': 847,
 'work': 828,
 'today': 822,
 'going': 799,
 'lol': 738,
 'one': 698,
 'time': 696,
 'im': 679,
 'know': 676,
 'back': 659,
 'u': 623,
 'really': 605,
 'amp': 573,
 'well': 570,
 'new': 567,
 'see': 563,
 'night': 559,
 'still': 547,
 'oh': 539,
 'thanks': 526,
 'want': 524,
 'think': 520,
 'na': 512,
 'home': 503,
 'much': 480,
 'need': 453,
 'miss': 446,
 'last': 445,
 'great': 433,
 'morning': 427,
 'tomorrow': 420,
 'feel': 396,
 'haha': 379,
 'twitter': 372,
 'bad': 363,
 'hope': 359,
 'sad': 359,
 'fun': 347,
 'wish': 343,
 'would': 341,
 'sleep': 337,
 'nice': 330,
 'sorry': 329,
 'make': 327,
 'right': 326,
 'happy': 322,
 'tonight': 319,
 'getting': 310,
 'come': 304,
 'gon': 299,
 'though': 298,
 'wait': 288,
 'better': 279,
 'bed': 277,
 'watching': 274,
 'way': 273,
 'hate': 269,
 'lt': 267,
 'yeah': 263,
 'week': 259,
 'people': 259,
 'could': 247,
 'thank': 242,
 

In [24]:
top_words= list(wordfreq.keys())[:5000]


In [25]:
top_words

['good',
 'day',
 'get',
 'like',
 'quot',
 'go',
 'love',
 'got',
 'work',
 'today',
 'going',
 'lol',
 'one',
 'time',
 'im',
 'know',
 'back',
 'u',
 'really',
 'amp',
 'well',
 'new',
 'see',
 'night',
 'still',
 'oh',
 'thanks',
 'want',
 'think',
 'na',
 'home',
 'much',
 'need',
 'miss',
 'last',
 'great',
 'morning',
 'tomorrow',
 'feel',
 'haha',
 'twitter',
 'bad',
 'hope',
 'sad',
 'fun',
 'wish',
 'would',
 'sleep',
 'nice',
 'sorry',
 'make',
 'right',
 'happy',
 'tonight',
 'getting',
 'come',
 'gon',
 'though',
 'wait',
 'better',
 'bed',
 'watching',
 'way',
 'hate',
 'lt',
 'yeah',
 'week',
 'people',
 'could',
 'thank',
 'next',
 'even',
 'little',
 'school',
 'working',
 'say',
 'everyone',
 'cant',
 'sick',
 'weekend',
 'show',
 'dont',
 'hey',
 'awesome',
 'days',
 'take',
 'yes',
 'life',
 'always',
 'never',
 'please',
 'soon',
 'watch',
 'first',
 'x',
 'long',
 'ok',
 'tired',
 'already',
 'wan',
 'friends',
 'sure',
 'best',
 'looking',
 'feeling',
 'man',
 'a

In [26]:
len(top_words)

5000

In [27]:
#word_tuples = [(word, frequency) for word, frequency in wordfreq.items()]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

In [28]:
data['is_positive'] = data['target'] == 4

In [29]:
data.head()

,target,id,date,flag,user,text,text_processed,is_positive
0,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,nkluvr eva my poor little dumpling in holmdel ...,"[nkluvr, eva, poor, little, dumpling, holmdel,...",False
1,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,i m off too bed i gotta wake up hella early to...,"[bed, got, ta, wake, hella, early, tomorrow, m...",False
2,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,i havent been able to listen to it yet my spea...,"[havent, able, listen, yet, speakers, busted]",False
3,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,"[remembers, solving, relatively, big, equation...",False
4,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,ate too much feel sick,"[ate, much, feel, sick]",False


In [44]:
data["text_processed"].head(5)

0    [nkluvr, eva, poor, little, dumpling, holmdel,...
1    [bed, got, ta, wake, hella, early, tomorrow, m...
2        [havent, able, listen, yet, speakers, busted]
3    [remembers, solving, relatively, big, equation...
4                              [ate, much, feel, sick]
Name: text_processed, dtype: object

In [47]:
def find_features(document, word_features):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [52]:
# featuresets = [(find_features(rev), category) for (rev, category) in data["text_processed"]]

featuresets = [(find_features(row.text,top_words), row.is_positive) for index, row in data.iterrows()]


In [53]:
featuresets[0]

({'good': False,
  'day': False,
  'get': False,
  'like': False,
  'quot': False,
  'go': False,
  'love': False,
  'got': False,
  'work': False,
  'today': False,
  'going': False,
  'lol': False,
  'one': False,
  'time': False,
  'im': False,
  'know': False,
  'back': False,
  'u': True,
  'really': False,
  'amp': False,
  'well': False,
  'new': False,
  'see': False,
  'night': False,
  'still': False,
  'oh': False,
  'thanks': False,
  'want': False,
  'think': False,
  'na': False,
  'home': False,
  'much': False,
  'need': False,
  'miss': False,
  'last': False,
  'great': False,
  'morning': False,
  'tomorrow': False,
  'feel': False,
  'haha': False,
  'twitter': False,
  'bad': False,
  'hope': False,
  'sad': False,
  'fun': False,
  'wish': False,
  'would': False,
  'sleep': False,
  'nice': False,
  'sorry': False,
  'make': False,
  'right': False,
  'happy': False,
  'tonight': False,
  'getting': False,
  'come': False,
  'gon': False,
  'though': False,
  'wa

In [ ]:
import pandas as pd
import numpy as np

# Sample DataFrame
#data = {'text_column': [['apple', 'banana', 'orange'], ['grape', 'kiwi'], ['apple', 'pear']]}
#df = pd.DataFrame(data)

# List of top words
# top_words = ['apple', 'banana', 'kiwi']

# Create a 2D array with boolean values indicating if top words are present in each list
result_dict = {word: [word in row for row in data['text_processed']] for word in top_words}

# Displaying the result
features= pd.DataFrame(result_dict)
features['is_positive'] = data['target'] == 4

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [63]:
from sklearn.model_selection import train_test_split
from nltk.classify import NaiveBayesClassifier

# set that we'll train our classifier with
train_set = featuresets[:16000]

# set that we'll test against.
test_set = featuresets[16000:]

classifier = NaiveBayesClassifier.train(train_set)
classifier.show_most_informative_features(100)

Most Informative Features
                       ½ = True             True : False  =      2.8 : 1.0
                       ï = True             True : False  =      2.6 : 1.0
                       ã = True            False : True   =      1.4 : 1.0
                       z = True             True : False  =      1.3 : 1.0
                       _ = True             True : False  =      1.3 : 1.0
                       x = True             True : False  =      1.2 : 1.0
                       q = True             True : False  =      1.2 : 1.0
                       l = False           False : True   =      1.2 : 1.0
                       â = True             True : False  =      1.2 : 1.0
                       j = True             True : False  =      1.2 : 1.0
                       h = False           False : True   =      1.1 : 1.0
                       n = False           False : True   =      1.1 : 1.0
                       b = False            True : False  =      1.1 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [60]:
from nltk.classify import accuracy
accuracy_score = accuracy(classifier, test_set)
print("Accuracy:", accuracy_score)

Accuracy: 0.539


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time.

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here